In [1]:
from seeq import spy
import pandas as pd

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.push

Uploads signals, conditions, scalars, and assets to the Seeq Server.

There are two main types of information processed by Seeq: _Data_ and _metadata_:

- **Data** is the time series and time interval information that is either collected or derived from sensor data. It consists of timestamps and values (samples), or time intervals and properties (capsules). This data can be plotted on a trend or used to train a neural network, for example.

- **Metadata** is the information about the data, that is independent of a particular point in time or time interval. For example, a signal's _name_, _description_ and _unit of measure_ is classified as metadata; or the formula that is used to derive a new signal from one or more source signals; or the asset tree that is used to model similar equipment or industrial processes.

The `spy.push()` command allows you to upload both types of information to Seeq Server. When you push _metadata_, you make entries in Seeq's data index, which allows you or other users to search for and find such entries. If you also push _data_, then samples or capsules or scalars will appear when the user selects those index entries for inclusion on a trend, scatter plot, scorecard or other visualization.

```
spy.push(data=None, metadata=None, item_type=None, workbook='Data Lab >> Data Lab Analysis',
         worksheet='From Data Lab', datasource=None, archive=False, type_mismatches='raise', errors='catalog')
```

## Workbook scoping

When you push any type of data, it is only available/discoverable within the workbook specified by the `workbook` argument. This allows you to _sandbox_ your activity by default, and only publish to your colleagues later when your experimentation is largely over.

## Pushing signal data

The simplest activity you can do with the `spy.push()` command is to read in a CSV file using Pandas and push it into Seeq. It will be stored in Seeq's internal time series database.

In [3]:
import csv
csv_file = pd.read_csv('csv_import_example.csv', parse_dates=['TIME(unitless)'], index_col='TIME(unitless)')
csv_file.head()

,BITDEP(ft),BLOCKCOMP(ft),DEP_RTN(ft),DEPTH(ft),FLOWIN(USgal/min),FLOWOUTPC(%),ROP_AVG(ft/h),DIFP(psi)
TIME(unitless),,,,,,,,
2018-07-25 23:00:01-06:00,4630.217,74.813,4627.0,4630.217,799.857,40.799,142.698,454.214
2018-07-25 23:00:03-06:00,4630.354,74.735,4627.5,4630.354,799.857,40.576,142.698,481.383
2018-07-25 23:00:05-06:00,4630.470,74.619,4627.5,4630.470,799.857,39.929,142.698,487.502
2018-07-25 23:00:07-06:00,4630.588,74.501,4627.5,4630.588,799.857,40.305,210.705,561.480
2018-07-25 23:00:09-06:00,4630.699,74.390,4627.5,4630.699,799.292,40.245,210.705,610.968


When you want to push data, it must have an index with a datetime data type. That's why we used the `parse_dates` and `index_col` arguments for `Pandas.read_csv()`.

Now you can just push it into Seeq:

In [4]:
push_results = spy.push(data=csv_file, workbook='SPy Documentation Examples >> spy.push')
push_results

,Count,Result,Time
BITDEP(ft),11485,Success,00:00:00.66
BLOCKCOMP(ft),11485,Success,00:00:01.16
DEP_RTN(ft),11485,Success,00:00:00.64
DEPTH(ft),11485,Success,00:00:00.73
FLOWIN(USgal/min),11485,Success,00:00:01.07
FLOWOUTPC(%),11485,Success,00:00:01.17
ROP_AVG(ft/h),11485,Success,00:00:01.08
DIFP(psi),11485,Success,00:00:01.03


,Push Count,Push Time,Push Result,Name,ID,Type
BITDEP(ft),11485.0,0:00:00.655002,Success,BITDEP(ft),BA4B6731-83B5-424B-BB96-15D459D82881,StoredSignal
BLOCKCOMP(ft),11485.0,0:00:01.157001,Success,BLOCKCOMP(ft),ED789F6A-5E4C-496D-A97B-2FC63E676620,StoredSignal
DEP_RTN(ft),11485.0,0:00:00.641000,Success,DEP_RTN(ft),BA1B39AF-9A8B-4065-A61A-95FC13476B27,StoredSignal
DEPTH(ft),11485.0,0:00:00.727998,Success,DEPTH(ft),522DF21C-0CA6-413B-A167-0551CB8DD681,StoredSignal
FLOWIN(USgal/min),11485.0,0:00:01.065000,Success,FLOWIN(USgal/min),04326EA0-4428-4A61-BE6A-F941818AF538,StoredSignal
FLOWOUTPC(%),11485.0,0:00:01.166990,Success,FLOWOUTPC(%),D5DECE75-A4D8-431C-9187-E52AEE7AAEC7,StoredSignal
ROP_AVG(ft/h),11485.0,0:00:01.078009,Success,ROP_AVG(ft/h),5B8933EE-0DE7-41A1-A7FA-B991EF28C975,StoredSignal
DIFP(psi),11485.0,0:00:01.028002,Success,DIFP(psi),F71AC208-A630-444B-8091-A1662A516A4D,StoredSignal


NOTE: Pushing data can be relatively slow. This is an area that Seeq will be optimizing in future versions.

You can push multiple times, and as long as the names are the same and the workbook hasn't changed, you'll just add to the existing signal.

## Pushing metadata

Now let's try pushing just metadata. You can see that the column names from the CSV file contain the unit of measure in parentheses. Let's use Pandas to extract the name and the unit of measure as separate columns.

In [5]:
better_metadata = push_results.copy()
better_metadata['Original Name'] = better_metadata.index
better_metadata['Name'] = better_metadata['Original Name'].str.extract(r'(.*)\(')
better_metadata['Value Unit Of Measure'] = better_metadata['Original Name'].str.extract(r'.*\((.*)\)')
better_metadata

,Push Count,Push Time,Push Result,Name,ID,Type,Original Name,Value Unit Of Measure
BITDEP(ft),11485.0,0:00:00.655002,Success,BITDEP,BA4B6731-83B5-424B-BB96-15D459D82881,StoredSignal,BITDEP(ft),ft
BLOCKCOMP(ft),11485.0,0:00:01.157001,Success,BLOCKCOMP,ED789F6A-5E4C-496D-A97B-2FC63E676620,StoredSignal,BLOCKCOMP(ft),ft
DEP_RTN(ft),11485.0,0:00:00.641000,Success,DEP_RTN,BA1B39AF-9A8B-4065-A61A-95FC13476B27,StoredSignal,DEP_RTN(ft),ft
DEPTH(ft),11485.0,0:00:00.727998,Success,DEPTH,522DF21C-0CA6-413B-A167-0551CB8DD681,StoredSignal,DEPTH(ft),ft
FLOWIN(USgal/min),11485.0,0:00:01.065000,Success,FLOWIN,04326EA0-4428-4A61-BE6A-F941818AF538,StoredSignal,FLOWIN(USgal/min),USgal/min
FLOWOUTPC(%),11485.0,0:00:01.166990,Success,FLOWOUTPC,D5DECE75-A4D8-431C-9187-E52AEE7AAEC7,StoredSignal,FLOWOUTPC(%),%
ROP_AVG(ft/h),11485.0,0:00:01.078009,Success,ROP_AVG,5B8933EE-0DE7-41A1-A7FA-B991EF28C975,StoredSignal,ROP_AVG(ft/h),ft/h
DIFP(psi),11485.0,0:00:01.028002,Success,DIFP,F71AC208-A630-444B-8091-A1662A516A4D,StoredSignal,DIFP(psi),psi


In [6]:
spy.push(metadata=better_metadata, workbook='SPy Documentation Examples >> spy.push')

,Push Count,Push Time,Name,ID,Type,Original Name,Value Unit Of Measure,Data ID,Datasource Class,Datasource ID,Push Result
BITDEP(ft),11485.0,0:00:00.655002,BITDEP,BA4B6731-83B5-424B-BB96-15D459D82881,StoredSignal,BITDEP(ft),ft,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
BLOCKCOMP(ft),11485.0,0:00:01.157001,BLOCKCOMP,ED789F6A-5E4C-496D-A97B-2FC63E676620,StoredSignal,BLOCKCOMP(ft),ft,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
DEP_RTN(ft),11485.0,0:00:00.641000,DEP_RTN,BA1B39AF-9A8B-4065-A61A-95FC13476B27,StoredSignal,DEP_RTN(ft),ft,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
DEPTH(ft),11485.0,0:00:00.727998,DEPTH,522DF21C-0CA6-413B-A167-0551CB8DD681,StoredSignal,DEPTH(ft),ft,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
FLOWIN(USgal/min),11485.0,0:00:01.065000,FLOWIN,04326EA0-4428-4A61-BE6A-F941818AF538,StoredSignal,FLOWIN(USgal/min),USgal/min,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
FLOWOUTPC(%),11485.0,0:00:01.166990,FLOWOUTPC,D5DECE75-A4D8-431C-9187-E52AEE7AAEC7,StoredSignal,FLOWOUTPC(%),%,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
ROP_AVG(ft/h),11485.0,0:00:01.078009,ROP_AVG,5B8933EE-0DE7-41A1-A7FA-B991EF28C975,StoredSignal,ROP_AVG(ft/h),ft/h,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success
DIFP(psi),11485.0,0:00:01.028002,DIFP,F71AC208-A630-444B-8091-A1662A516A4D,StoredSignal,DIFP(psi),psi,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Signal...,Seeq Data Lab,Seeq Data Lab,Success


## Pushing condition and capsule data

You can also push capsules, which are time intervals of interest, to Seeq by supplying a DataFrame with `Capsule Start` and `Capsule End` columns. Any additional columns will be added as properties of the capsule.

In [7]:
capsule_data = pd.DataFrame([{
    'Capsule Start':    pd.to_datetime('2019-01-10T09:00:00.000Z'),
    'Capsule End':      pd.to_datetime('2019-01-10T17:00:00.000Z'),
    'Operator On Duty': 'Mark'
}, {
    'Capsule Start':    pd.to_datetime('2019-01-11T09:00:00.000Z'),
    'Capsule End':      pd.to_datetime('2019-01-11T17:00:00.000Z'),
    'Operator On Duty': 'Hedwig'
}])

capsule_data

,Capsule Start,Capsule End,Operator On Duty
0,2019-01-10 09:00:00+00:00,2019-01-10 17:00:00+00:00,Mark
1,2019-01-11 09:00:00+00:00,2019-01-11 17:00:00+00:00,Hedwig


When you push capsule data, you must supply a `metadata` DataFrame that is exactly 1 row and contains, at minimum, the `Name`, `Type` and `Maximum Duration` columns like so:

In [8]:
spy.push(data=capsule_data,
         metadata=pd.DataFrame([{
             'Name': 'Operator Shifts',
             'Type': 'Condition',
             'Maximum Duration': '2d'
         }]), workbook='SPy Documentation Examples >> spy.push')

,ID,Name,Count,Time,Result
0,F79F033D-7C52-4463-953F-2AAA941092BE,Operator Shifts,2,00:00:00.05,Success


,Name,Type,Maximum Duration,Datasource Class,Datasource ID,Data ID,ID,Push Result,Push Count,Push Time
0,Operator Shifts,StoredCondition,2d,Seeq Data Lab,Seeq Data Lab,[32CE3D8A-DD48-4CE6-9864-AC0EDEA46DD9] {Condit...,F79F033D-7C52-4463-953F-2AAA941092BE,Success,2,0:00:00.048998


Capsule properties that have units of measure can be specified in the metadata. Say you have properties like `Height` and `Mass`:

In [3]:
capsule_data = pd.DataFrame([{
    'Capsule Start':    pd.to_datetime('2019-01-10T09:00:00.000Z'),
    'Capsule End':      pd.to_datetime('2019-01-10T17:00:00.000Z'),
    'Height': 5,
    'Mass': 10,
}, {
    'Capsule Start':    pd.to_datetime('2019-01-11T09:00:00.000Z'),
    'Capsule End':      pd.to_datetime('2019-01-11T17:00:00.000Z'),
    'Height': 3,
    'Mass' : 6
}])

capsule_data

,Capsule Start,Capsule End,Height,Mass
0,2019-01-10 09:00:00+00:00,2019-01-10 17:00:00+00:00,5,10
1,2019-01-11 09:00:00+00:00,2019-01-11 17:00:00+00:00,3,6


Use `Capsule Property Units` in metadata dataframe to specify that `Height` has units of meters, `m` and Mass has units of `kg` :

In [4]:
spy.push(data=capsule_data,
         metadata=pd.DataFrame([{
             'Name': 'In Production',
             'Type': 'Condition',
             'Maximum Duration': '2d',
             'Capsule Property Units': {'Height': 'm',
                                        'Mass': 'kg'}
         }]), workbook='SPy Documentation Examples >> spy.push')

,ID,Name,Count,Pages,Time,Result
0,B30132BE-6A19-4EE3-B1A1-837DD1E6E297,In Production,2,1,00:00:00.02,Success


,Name,Type,Maximum Duration,Capsule Property Units,Datasource Class,Datasource ID,Data ID,ID,Push Result,Push Count,Push Time
0,In Production,StoredCondition,2d,"{'Height': 'm', 'Mass': 'kg'}",Seeq Data Lab,Seeq Data Lab,[D1447C9E-9415-474F-8009-2F03501766A6] {Condit...,B30132BE-6A19-4EE3-B1A1-837DD1E6E297,Success,2,0:00:00.016864


## Updating capsules

To update capsules, you must replace them with new capsules. An easy workflow for this is to pull all the capsules in the range you want to edit, make changes, and then push with the `replace` argument set to the same range.


First, get the condition using `spy.search()`:

In [5]:
condition = spy.search({'Name': 'In Production'}, workbook='SPy Documentation Examples >> spy.push')
condition

,Name,Time,Count,Pages,Result
0,In Production,00:00:00.02,1,1,Success


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,B30132BE-6A19-4EE3-B1A1-837DD1E6E297,In Production,NaN,StoredCondition,NaN,Seeq Data Lab,False


The start and end should be identical for both the `spy.pull()` and the `replace` argument to `spy.push()` so that nothing is accidentally deleted or duplicated. To make this easier, save them as variables.

In [6]:
replace_start=pd.to_datetime('2019-01-10T09:00:00.000Z')
replace_end=pd.to_datetime('2019-01-11T17:00:00.000Z')

Get all the capsules within that range:

In [7]:
new_capsule_data = spy.pull(condition, 
                    start=replace_start, 
                    end=replace_end)
new_capsule_data

,ID,Name,Time,Count,Pages,Result
0,B30132BE-6A19-4EE3-B1A1-837DD1E6E297,In Production,00:00:00.10,2,1,Success


,Condition,Capsule Start,Capsule End,Capsule Is Uncertain,Mass,Height
0,In Production,2019-01-10 09:00:00+00:00,2019-01-10 17:00:00+00:00,False,10,5
1,In Production,2019-01-11 09:00:00+00:00,2019-01-11 17:00:00+00:00,False,6,3


Now we can update specific capsule properies. Let's say the height of the first capsule was actually 4 meters and the Mass for the second capsule is 7 kg :

In [8]:
new_capsule_data.at[0, 'Height'] = 4
new_capsule_data.at[1, 'Mass'] = 7
new_capsule_data

,Condition,Capsule Start,Capsule End,Capsule Is Uncertain,Mass,Height
0,In Production,2019-01-10 09:00:00+00:00,2019-01-10 17:00:00+00:00,False,10,4
1,In Production,2019-01-11 09:00:00+00:00,2019-01-11 17:00:00+00:00,False,7,3


Additional capsule propeties can also be added at this time. Suppose you want to record what the temperature was for the first capsule. A new `Temperature` property can be added:

In [9]:
new_capsule_data.at[0, 'Temperature'] = 67
new_capsule_data

,Condition,Capsule Start,Capsule End,Capsule Is Uncertain,Mass,Height,Temperature
0,In Production,2019-01-10 09:00:00+00:00,2019-01-10 17:00:00+00:00,False,10,4,67.0
1,In Production,2019-01-11 09:00:00+00:00,2019-01-11 17:00:00+00:00,False,7,3,NaN


You must specify units of measure for existing properties (e.g., `Height` and `Mass`) with every `spy.push()` along with units for new properties (e.g., `Temperature` measured in `F`). 

The `replace` parameter takes a dictionary with the keys `'Start'` and `'End'`. Any capsules that start in the provided time period will be replaced. 

Push the `new_capsule_data` using the `replace` parameter.

In [10]:
spy.push(data=new_capsule_data,
         metadata=pd.DataFrame([{
             'Name': 'In Production',
             'Type': 'Condition',
             'Maximum Duration': '2d',
             'Capsule Property Units': {'Height':'m',
                                        'Mass':'kg', 
                                        'Temperature':'F'}
         }]), 
         replace={'Start':   replace_start,
                  'End':     replace_end},
         workbook='SPy Documentation Examples >> spy.push')

,ID,Name,Count,Pages,Time,Result
0,B30132BE-6A19-4EE3-B1A1-837DD1E6E297,In Production,2,1,00:00:00.10,Success


,Name,Type,Maximum Duration,Capsule Property Units,Datasource Class,Datasource ID,Data ID,ID,Push Result,Push Count,Push Time
0,In Production,StoredCondition,2d,"{'Height': 'm', 'Mass': 'kg', 'Temperature': 'F'}",Seeq Data Lab,Seeq Data Lab,[D1447C9E-9415-474F-8009-2F03501766A6] {Condit...,B30132BE-6A19-4EE3-B1A1-837DD1E6E297,Success,2,0:00:00.099164


## Deleting capsules

The `replace` argument can also be used to delete capsules. All capsules starting within the time range specified by `replace` are deleted before the new capsules are added, so providing no new capsules to `spy.push()` functions as a simple deletion.

<b>Caution</b>: any capsules whose `Start` value is within the `replace` range will be deleted. The `replace` `Start` is inclusive and `replace` `End` is exclusive.

To delete the first capsule, we'll specify a time range that includes its start (and no other capsule's start):

In [ ]:
spy.push(metadata=pd.DataFrame([{
             'Name': 'In Production',
             'Type': 'Condition',
             'Maximum Duration': '2d'}]),
         replace={'Start':    pd.to_datetime('2019-01-10T09:00:00.000Z'),
                  'End':      pd.to_datetime('2019-01-10T10:00:00.000Z')},
         workbook='SPy Documentation Examples >> spy.push')

If there are several capsules that start at the same time or within the same range, you can delete just one by following the update workflow but removing the capsule to be deleted from the `pandas` DataFrame before using `spy.push()`

## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cell below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [9]:
help(spy.push)

Help on function push in module seeq.spy._push:

push(data=None, *, metadata=None, replace=None, workbook='Data Lab >> Data Lab Analysis', worksheet='From Data Lab', datasource=None, archive=False, type_mismatches='raise', errors='raise', quiet=False, status=None, session: Union[seeq.spy._session.Session, NoneType] = None)
    Imports metadata and/or data into Seeq Server, possibly scoped to a
    workbook and/or datasource.
    
    The 'data' and 'metadata' arguments work together. Signal and condition
    data cannot be mixed together in a single call to spy.push().
    
    Successive calls to 'push()' with the same 'metadata' but different 'data'
    will update the items (rather than overwrite them); however, pushing a new
    sample with the same timestamp as a previous one will overwrite the old
    one.
    
    Metadata can be pushed without accompanying data. This is common after
    having invoked the spy.assets.build() function. In such a case, the
    'metadata' DataFrame